In [ ]:
# %pip install opencv-python-headless numpy scikit-learn tensorflow

In [ ]:
# Restart the runtime
# !kill -9 -1

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [2]:
# 1. Import Libraries
import os

import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# 2. Dataset Loading

dataset_path = (
    "/content/drive/MyDrive/Projects/Plant Disease Prediction/Groundnut_Leaf_dataset"
)
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")

# Get the class names (disease names)
class_names = os.listdir(train_dir)
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

In [4]:
# 3. Data Preprocessing

images = []
labels = []

for split_dir in [train_dir, test_dir]:
    for class_name in class_names:
        class_dir = os.path.join(split_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith((".jpg", ".png", ".jpeg")):
                img_path = os.path.join(class_dir, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (224, 224))  # Resize

                    height, width = img.shape[:2]
                    size = min(height, width)
                    x = (width - size) // 2
                    y = (height - size) // 2
                    img = img[y : y + size, x : x + size]

                    original_shape = img.shape
                    image_2d = img.reshape(-1, 3)
                    scaler = MinMaxScaler(feature_range=(0, 1))
                    normalized_2d = scaler.fit_transform(image_2d)
                    normalized_image = normalized_2d.reshape(original_shape)

                    images.append(normalized_image)
                    labels.append(class_name)
                else:
                    print(f"Error reading image: {img_path}")

images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)

In [ ]:
# 4. Label Encoding
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
print(labels_encoded)
print(set(labels_encoded))

In [ ]:
# 5. Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

print("Train data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(224, 224, 3)
        ),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
# 7. Model Training
epochs = 10
batch_size = 32

history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
)

In [ ]:
# 8. Model Evaluation:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
# 9. Plotting Training History

plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()